# Random Forest Classifier

**Author: Jessica Cervi**

**Expected time = 2 hours**

**Total points = 75 points**
 
 
 
## Assignment Overview

This assignment is designed to build your familiarity and comfort coding in Python while also helping you review key topics from each module. As you progress through the assignment, answers will get increasingly complex. You must adopt a data scientist's mindset when completing this assignment. **Remember to run your code from each cell before submitting your assignment.** Running your code beforehand will notify you of errors and give you a chance to fix your errors before submitting it. You should view your Vocareum submission as if you are delivering a final project to your manager or client. 

***Vocareum Tips***
- Do not add arguments or options to functions unless asked specifically. This will cause an error in Vocareum.
- Do not use a library unless you are explicitly asked in the question.
- You can download the Grading Report after submitting the assignment. It will include the feedback and hints on incorrect questions. 


### Learning Objectives

- Examine the class distribution in a classification problem 
- Handle the categorical features using "get dummies" 
- Investigate class splits using the Gini index
- Determine baseline accuracy for classifiers 
- Organize grid search hyperparameters in tree models 



## Index: 

#### Random Forest Classifier

- [Question 1](#q1)
- [Question 2](#q2)
- [Question 3](#q3)
- [Question 4](#q4)
- [Question 5](#q5)
- [Question 6](#q6)
- [Question 7](#q7)
- [Question 8](#q8)
- [Question 9](#q9)
- [Question 10](#q10)
- [Question 11](#q11)






This assignment uses a variety of decision tree based classifiers to attempt prediction of whether or not a customer will default on their loans. Our data comes from [the UCI machine learning Repository](https://archive.ics.uci.edu/ml/index.php) on [default of credit card clients](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#)  

Activities will include:  
- Manipulating DataFrames  
- Visualizing data  
- Calculating impurity measures  
- Using sklearn's tree and forest models  
- Evaluating the effects of hyperparameter tuning  


**Motivation**: Decision Trees and Forests offer easy to understand yet fairly advanced models with a variety of hyperparameters to increase or decrease complexity to tune between bias and variance


## Random Forest Classifier




### Importing the data set and exploratory data analysis

This assignment uses a variety of decision tree-based classifiers to attempt a prediction of whether or not a customer will default on their loans. Our data comes from [the UCI machine learning Repository](https://archive.ics.uci.edu/ml/index.php) on [default of credit card clients](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#).

In particular, using a given a number of personal variables - sex, education, marriage status, age; and recent payment history, we will attempt to predict whether or not a customer will default in the next month.  

Before coding an algorithm, we will take a look at our data using 'Python's 'pandas'. For visualizations, we'll use 'matplotlib'.

Let's import the necessary libraries and load the datasets. We will be using the pandas 'pd.read_excel()' function. 

In [ ]:
# Import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


# Read in Data
df = pd.read_excel("./data/credit_card_data.xls", header = 1)

df.rename(columns = {"PAY_0":"PAY_1"}, inplace = True) #renaming mis-named column

We begin by performing some basic exploratory data analysis by using the function 'head()' and the attribute 'columns'.

In [ ]:
df.head()

 

- X1: Amount of the given credit (NT dollar): it includes both, the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. 

Next, it is always important to check whether we have any null values on our dataframe. We can do so by using the command '.info()'.

In [ ]:
print("Data Shape: " , df.shape, "\n")
df.info()

Great, it appears that there aren't any null in our dataframe.
   

[Back to top](#Index:) 
<a id='q1'></a>

### Question 1:

*5 points*

The 'default payment next month' variable will be the target used for classification. Investigate its distribution for the question below.

Assign an int to the variable 'ans0' corresponding to the total number of non-default records.
Assign an int to the variable 'ans1' corresponding to the total number of default records

Note a "0" in the 'default payments next month' column means "non-default" and a 1 means "default".

In [ ]:
### GRADED



### YOUR ANSWERS BELOW

ans0 = None
ans1 = None

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


We observe that the proportion of default and non-default records means we are dealing with unbalanced classes.

#### Investigating the Features

Now, we will investigate some features trends using the function 'value_count()'.

In [ ]:
df['EDUCATION'].value_counts()

Note: The code-book from where we have downloaded the dataset, describes the education variable having only four values (1-4). However, there are seven values (0-6).  
In some cases, this might be grounds to throw out the unknown values 0,5, and 6. For now, we will leave them as is, assuming that they have some (unknown to us) meaning.

In [ ]:
df['MARRIAGE'].value_counts()

Note: Again, the code-book describes only three values for marriage (1-3). However, here, "0" also appears. Given what we saw above, we might assume that "0" in these categorical variables is functionally the "null" value.

In [ ]:
df['SEX'].value_counts()

Note: A slight imbalance exists in the representation of men and women, with women making up a little over 60% of our observations. 

In [ ]:
df['PAY_1'].value_counts()

#### Investigate relationship between the variables - 'Pay', 'Bill_amt', and 'Pay_amt'.

In [ ]:
for i in [-2,-1,0,1,2,8]:
    print(df[df['PAY_1']==i][['PAY_1','BILL_AMT1','PAY_AMT1']].head(8), "\n")

It is unclear exactly how these 'PAY_1','BILL_AMT1', and 'PAY_AMT1' variables work and how they are related to each other. Possibly they should be treated as categorical data instead of discrete and interval data, but, for our purposes, we keep them as interval data.

Below, we define a function to create histograms for the 'PAY_1','BILL_AMT1', and 'PAY_AMT1' variables.

In [ ]:
### Define function for creating histograms
def pay_hist(df, cols, ymax):
    plt.figure(figsize= (10,7)) # define fig size
    
    for index, col in enumerate(cols): # For each column passed to function
        plt.subplot(2,3, index +1) # plot on new subplot
        plt.ylim(ymax = ymax) # standardize ymax
        plt.hist(df[col]) # create hist
        plt.title(col) # title with column names
    plt.tight_layout(); # make sure titles don't overlap

pay_cols = ["PAY_"+str(n) for n in range(1,7)]
pay_amt_cols = ['PAY_AMT' + str(n) for n in range(1,7)]
bill_amt_cols = ['BILL_AMT' + str(n) for n in range(1,7)]

In [ ]:
pay_hist(df, pay_cols, 20000)

Note: Clearly, "0" is the majority class for all of the "PAY" variables. However, we recall that from the code-book, it is unclear what "0" means.

In [ ]:
pay_hist(df, pay_amt_cols, 20000)

In [ ]:
df_no_0_pay_amt_1 = df[df["PAY_AMT1"]!=0]
df_no_0_pay_amt_1["PAY_AMT1"].hist()

We notice that even taking out all the PAY_AMT of 0, most payments stay close to 0 with a long tail.

When dealing with skewed data, log transformation (used above) or $\sqrt[4]{x}$ transformations (which automatically deals with 0s), can become useful. Below, we see that most of the repayment amounts are in the thousands of dollars after using log transformation.

In [ ]:
log_pay_amt1 = np.log10(df_no_0_pay_amt_1["PAY_AMT1"])
plt.hist(log_pay_amt1)
plt.title("Log10-Transformed values for 'PAY_AMT1' (Excluding 0s)");

In [ ]:
pay_hist(df, bill_amt_cols, 23000)

### Preprocessing

Currently, the coding used in the variable 'SEX' is 2 for 'female' and 1 for 'male'. We want 'SEX' to be in binary code. So, we will change the column name to 'FEMALE' and subtract 1 from each entry. This way, 1 will correspond to 'female' and 0 to 'male'.

[Back to top](#Index:) 
<a id='q2'></a>

### Question 2:

*5 points*

Change the values of the column 'SEX' to 0 and 1. Next, rename the column 'SEX' to 'FEMALE' and the column 'default payment next month' to 'default'.

In [ ]:
### GRADED



### YOUR ANSWERS BELOW


###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Next, we observe that both 'EDUCATION' and 'MARRIAGE' are categorized with multiple options. The function 'pd.get_dummies()' will allow us to create n-1 binary features to encode the n categories.  



#### Note regarding `get_dummies()`

Always be careful in using the function 'pd.get_dummies()'. It is NOT appropriate in many machine learning applications.  

Example: Currently we have the values 0-6 under the 'EDUCATION' feature. Running the  function 'pd.get_dummies()' on the test set would add 6 columns. Instead, the training set would only have 5 columns added from that action. Therefore, the fit model would not know how to deal that new feature/category.  

Here, the function 'pd.get_dummies()'used for the sake of simplicity.

[Back to top](#Index:) 
<a id='q3'></a>

### Question 3:

*10 points*

Rename the columns 'EDUCATION' and 'MARRIAGE' as 'EDU' and 'MAR', respectivly. Next, encode 'EDUCATION' AND 'MARRIAGE'  by using the function 'get_dummies()' and remove the original 'EDUCATION' and 'MARRIAGE' columns.

**HINT:** In 'get_dummies()', set 'prefix = pre, drop_first = True'.

In [ ]:
### GRADED



### YOUR ANSWERS BELOW


###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q4'></a>

### Question 4:

*5 points*

Why are 'dummy' variables necessary?

- a) It helps to expand the number of features.
- b) It deals with multi-class categorical data unlike models.
- c) It translates categorical data into quantitative data.

Assign a character corresponding to your choice as string to 'ans4'.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans4 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


<a id = "part2"></a>
## Trees and Forests: Introduction


Given a target variable (categorical or continuous), a decision tree iteratively splits the data at the value that creates the greatest separation among the target variable.

For example, let the target variable on a dataset of professional basketball players is league: NBA and WNBA. The available features are height and weight.  

Split 1: For the players greater than 6'8" tall (which is very rare), a decision tree might split the data at the height of 6'8", 6'7", or 6'6" and predict for a basketball player over 6'6", plays in the NBA.  

Split 2: For the players shorter than 6'6", the decision tree might split the data again at 5'10" and predict for the basketball players under 5'10", play in the WNBA.  

Split 3: For the players between 5'10" and 6'6", maybe the tree might discriminate on weight, predicting that all those players who weigh more than 180 Lbs., play in the NBA.  

The resulting tree could be visualized as below.

![DTExample](./assets/DTExample.png)

### Depth

The above tree has a depth of 2. 

The maximum depth to which a tree is allowed to grow can be specified using the function 'max_depth' from 'sklearn'. By default, 'max_depth' is set to 'None' that means the tree will grow until all leaves (terminal nodes) are pure, or until other user-specified criteria are met. Importantly, the function 'max_depth' can impact the amount of time it takes to build a tree (this becomes especially important when starting to work with forests.)  

### Splitting

The splits in the above trees were determined intuitively. Thankfully, the decision trees do not make their decisions using intuition. In the 'sklearn' package, two splitting criteria are available for classifiers-'Gini' and 'Entropy'. In general, 'Gini' favors splitting larger partitions, whereas 'Entropy' favors splitting of smaller groups that are of a single class. 

For more information about the 'Gini' and the 'Entropy' classifiers visit: [More on Gini/Entropy](http://www.learnbymarketing.com/481/decision-tree-flavors-gini-info-gain/)

### Gini and Entropy Calculations

<table><tr><td><img src='./assets/choc.png' style="width: 70%;"></td><td><img src='./assets/choc2.png' style="width: 70%;"></td></tr></table>  


Above, we have split the dataset regarding good chocolate and bad chocolate.  

In the next question, you will be asked to calculate the Gini impurity.

When 'American == True', we have 175 good chocolates and 330 bad chocolates. When 'American == False', there are 200 good chocolates and 120 bad chocolates.

When 'German == True', we have 175 good chocolates and 330 bad chocolates. When 'German == False', there are 200 good chocolates and 120 bad chocolates.

As a reminder, the Gini index for a node can be computed via:

$$1- \sum_{j=1}^n p^2_j,$$
where $n$ are the classes and $p_j$ is the frequency of class j in that node.  

Finally, the indexes for each of these nodes are weighted by the proportion of data at each node, then summed.  
Remember, Gini indicies closer to 0 are purer.

[Back to top](#Index:) 
<a id='q5'></a>

### Question 5:

*10 points*

Calculate the Gini impurity for the split on 'American'.

Assign float answer to the variable 'ans5',  round your answer to 4 decimal places.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans5 = None

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q6'></a>

### Question 6:

*10 points*

Calculate the Gini impurity for the split on 'German'.

Assign float answer to the variable 'ans6',  round your answer to 4 decimal places.

In [ ]:
### GRADED



### YOUR ANSWER BELOW

ans6 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


[Back to top](#Index:) 
<a id='q7'></a>

### Question 7:

*5 points*

According to the Gini values calcualted above, which split is better?

- a) American
- b) German
Assign character associated with your choice as string to 'ans7'.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans7 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Forests

'Forests' are the collections of decision trees that are designed to protect against over-fitting.  

A single decision tree (particularly one that is allowed to grow to any depth), may be prone to overfitting. Algorithmically, the tree is designed to continue to make splits until it has completely classified all of the available data, and/or exhausted every possible split for some very high level of complexity. A tree might be 'pruned' (by setting max_depth) to protect against over fitting, but, we can use a 'forest' of trees instead.  

### Baseline accuracy

[Back to top](#Index:) 
<a id='q8'></a>

### Question 8:

*5 points*

Going back to our credit card example, assume we are predicting default/not-default.
What percent (between 0 and 100) of observations would be correctly predicted if the majority class was predicted every time? In other words, what is the baseline accuracy?

Assign integer value to 'ans8'.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans8=None 

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Decision trees in sklearn

Like all other machine learning algorithms that we have seen so far, decision trees are also implemented automatically in 'sklearn'.

For a more complete description of how to implement decision trees  in `sklearn` you can visit the [decision trees in `sklearn` - Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)  


[Back to top](#Index:) 
<a id='q9'></a>

### Question 9:

*10 points*


Split our data into `X_train`, `X_test`, `y_train` and `y_test` sets by setting, as target, the column `default`. Set the `test_size =.3` and  `random_state = 1738`.

Use the function `DecisionTreeClassifier` from `sklearn` to instantiate the classifier `dt`.

Use the function `BaggingClassifier` from `sklearn` to instantiate the classifier `bag`.

Use the function `RandomForestClassifier` from `sklearn` to instantiate the classifier `rf`.

Use the function `ExtraForestClassifier` from `sklearn` to instantiate the classifier `et`.


In [ ]:
### GRADED

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### YOUR ANSWER BELOW
X_train = None
X_test = None
y_train = None
y_test = None
dt = None
bag = None
rf = None
et = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Next, we look at performance of the different classifiers using the default parameters.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

dt.fit(X_train, y_train)
print("Decision Tree: \n", classification_report(y_test, dt.predict(X_test)), "\n")


print("-----------")

bag.fit(X_train, y_train)
print("Bagging: \n", classification_report(y_test, bag.predict(X_test)), "\n")



print("-----------")

rf.fit(X_train, y_train)
print("Random Forest: \n", classification_report(y_test, rf.predict(X_test)), "\n")

print("------------")

et.fit(X_train, y_train)
print("Extra Trees: \n", classification_report(y_test, et.predict(X_test)), "\n")



[Back to top](#Index:) 
<a id='q10'></a>

### Question 10:

*5 points*

Are all predictions in this dataset equal in business value? In other words, is it correct to predict a 'default' prediction as valuable as a 'non-default' one?
- a) Equal values
- b) Unequal values

Assign string associated with your choice to 'ans9'.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans9 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Hyper Parameter Tuning

It appears that both the Random Forest and the Extra Trees perform in a similarly effective manner. For this section, we will try to increase the performance of the Random Forest by tuning a couple of hyperparameters, namely, criterion and estimators.

We are more interested in forecasting defaults than non-defaults. So, we will optimize the recall of defaults, where recall is the proportion of defaults predicted over total defaults.  

The code cell below will not run on Vocareum due to processing constraints, thus the output is copied below.

In [ ]:
# %%time
# from sklearn.metrics import recall_score
# criterion = ['gini', 'entropy']
# n_estimators = [5,10,20, 50, 100]
# scores = dict()
# i = 0
# for c in criterion:
#     for e in n_estimators:
#         rf = RandomForestClassifier(n_estimators = e, criterion = c, random_state = 1738)
#         rf.fit(X_train, y_train)
#         scores[i] = {'recall':recall_score(y_test, rf.predict(X_test)), 'trees' :e, "crit":c}
#         i+=1

In [ ]:
# pd.DataFrame(scores).T

![recallScores](./assets/recallScores3.PNG)

[Back to top](#Index:) 
<a id='q11'></a>

### Question 11:

*5 points*

According to the output shown in the picture above, how many trees did the model with the best recall score featured? Assign integer value to the variable 'ans10'.

Which splitting method?
- a) entropy 
- b) gini
Assign the character corresponding to your selection as a string to the variable 'ans11'.

In [ ]:
### GRADED


### YOUR ANSWER BELOW

ans10 = None
ans11 = None
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


We suspect that 'RandomForestClassifier' will perform best using 'Gini' splittling with somewhere around 5 trees.

Let's see what happens by tuning some paramenters.

In [ ]:
# n_estimators = [1,2,3,4,5,6,7,8]
# scores2 = dict()
# i = 0
# for e in n_estimators:
#     rf = RandomForestClassifier(n_estimators = e, criterion = 'gini', random_state = 1738)
#     rf.fit(X_train, y_train)
#     scores2[i] = {'recall':recall_score(y_test, rf.predict(X_test)), 'trees' :e}
#     i+=1

In [ ]:
# pd.DataFrame(scores2).T

![recallScores2](./assets/recallScores4.PNG)

We are focussing on Gini random forests with trees between 1 and 8. There is not enough consistency in the results to say more trees are better or worse. However, it might be tempting to try to tune-and-tune hyperparameters to increase scores. In many cases (as in the example above), mostly hyperparameter tuning results in not creating a better model, but it does a better job of predicting the test set.